# Generate Credit Card Transactions
**This notebook generates credit card transactions and randomly injects fraud chain attacks.**


---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Generate Transactions](#Generate-Transactions)
1. [Inject Fradulent Transactions](#Inject-Fradulent-Transactions)
1. [Save Generated Data](#Save-Generated-Data)

### Background

This notebook generates random credit card transactions for some number of unique users over a period of one or more months. In an ideal scenario, these historical transactions would be accumulated into a data lake/store for batch processing so as to derive insights and analytics about this data. Credit card numbers can be bought in bulk on the dark web through previous leaks or hacks of organizations that store this sensitive data. Fraudsters will buy these card lists and attempt to make as many transactions as possible with the stolen numbers until the card is blocked. These fraud chain attacks typically happen in a short time frame and can be easily spotted amongst historical transactions. This is because the velocity of transactions during the attack significantly differs from that of cardholder’s usual spending pattern. This notebook is optional to run. The generated data already exists in the `./data` folder for you to use. Re-run this notebook if you desire to re-populate fresh data or understand the whole process of how this dataset was generated.

We use the python Faker library to generate realistic character strings that represent items like credit card numbers, consumer IDs, etc. Faker library documentation: https://faker.readthedocs.io/en/master/

Usage Notes:
If you want to tailor the counts in order to create larger or smaller datasets, you can adjust these variables in the code cell below: `TOTAL_UNIQUE_TRANSACTIONS`, `TOTAL_UNIQUE_CREDIT_CARDS`, `TOTAL_UNIQUE_CONSUMERS`



### Setup

#### Prerequisites 

In [ ]:
!pip install Faker

#### Imports 

In [ ]:
from botocore.client import ClientError
from collections import defaultdict
from faker import Faker
import pandas as pd
import numpy as np
import sagemaker
import datetime
import hashlib
import random
import boto3
import math
import os

#### Seed for Reproducibility

In [ ]:
faker = Faker()
faker.seed_locale('en_US', 0)

In [ ]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants 

In [ ]:
# Define counts
TOTAL_UNIQUE_TRANSACTIONS = 100 * 1000
TOTAL_UNIQUE_CREDIT_CARDS = 2000
TOTAL_UNIQUE_CONSUMERS = 1000

BUCKET = sagemaker.Session().default_bucket()

### Generate Transactions

#### Generate Unique Credit Card Numbers 
<p> Credit card numbers are uniquely assigned to users. Number of users to generate is passed in as argument 'n'. </p>

In [ ]:
def generate_unique_credit_card_numbers(n: int) -> list:
    cc_ids = set()
    for _ in range(n):
        cc_id = faker.credit_card_number(card_type='visa')
        cc_ids.add(cc_id)
    return list(cc_ids) 

In [ ]:
credit_card_numbers = generate_unique_credit_card_numbers(TOTAL_UNIQUE_CREDIT_CARDS)

In [ ]:
assert len(credit_card_numbers) == TOTAL_UNIQUE_CREDIT_CARDS 
assert len(credit_card_numbers[0]) == 16 # validate if generated number is 16-digit

In [ ]:
# inspect random sample of credit card numbers 
random.sample(credit_card_numbers, 5)

#### Generate Consumer IDs
Since we are training a consumer fraud detection model, we need a set of unique consumer IDs that we can randomly associate with each credit card transaction. The relationship between Consumer IDs to Credit Cards is one-to-many; a single Consumer can be tied to one or more credit instruments.

In [ ]:
# We use Faker's Basic Bank Account Number (bban) module as a proxy for Consumer ID
def generate_unique_consumer_ids(n: int) -> list:
    id_list = set()
    for _ in range(n):
        bban_id = faker.unique.bban()
        id_list.add(bban_id)
    return list(id_list) 

In [ ]:
consumer_ids = generate_unique_consumer_ids(TOTAL_UNIQUE_CONSUMERS)

In [ ]:
assert len(consumer_ids) == TOTAL_UNIQUE_CONSUMERS

In [ ]:
random.sample(consumer_ids, 5)

#### Generate Time Series
<p>Generate unique random timestamps spread across a time period specified below (e.g. one month, 2021-03-01 to 2021-03-31, in sorted order. The number of timestamps to generate is passed in as argument. </p>
<b>Note:</b> The timestamps are NOT unique themselves. We can have 2 or more transactions occurring at the same time coming from different users. 

In [ ]:
# Feature Store Group requires ISO-8601 string format: yyyy-MM-dd'T'HH:mm:ssZ
# when the EventTime required attribute is type String

ISO_8601_DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

In [ ]:
def generate_timestamps(n: int) -> list:
    # Use timeframe of one month Mar 2021   
    start = datetime.datetime.strptime('2021-03-01T00:00:00Z', ISO_8601_DATETIME_FORMAT)
    end = datetime.datetime.strptime('2021-03-31T23:59:59Z', ISO_8601_DATETIME_FORMAT)

    timestamps = list()
    for _ in range(n):
        # We use .isoformat() to convert datetime to ISO-8601 Format, required by Feature Store
        iso_timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime(ISO_8601_DATETIME_FORMAT)
        timestamps.append(iso_timestamp)
    timestamps = sorted(timestamps)
    return timestamps

In [ ]:
timestamps = generate_timestamps(TOTAL_UNIQUE_TRANSACTIONS)

In [ ]:
assert len(timestamps) == TOTAL_UNIQUE_TRANSACTIONS

In [ ]:
# inspect random sample of timestamps
random.sample(timestamps, 5)

#### Generate Random Transaction Amounts 
<p>The transaction amounts are presumed to follow Pareto distribution, as it is logical for consumers to make many more smaller purchases than large ones. The break down of the distribution is shown in the table below.</p>


| Percentage        | Range (Amount in $)     |
| :-------------: | :----------: |
|  5\% | 0.01 to 1    |
| 7.5\%   | 1 to 10 |
| 52.5\%   | 10 to 100 |
| 25\%   | 100 to 1000 |
| 10\%   | 1000 to 10000 |

In [ ]:
def get_random_transaction_amount(start: float, end: float) -> float:
    amt = round(np.random.uniform(start, end), 2)
    return amt

In [ ]:
distribution_percentages = {0.05: (0.01, 1.01), 
                            0.075: (1, 11.01),
                            0.525: (10, 100.01),
                            0.25: (100, 1000.01),
                            0.10: (1000, 10000.01)}

In [ ]:
amounts = []

for percentage, span in distribution_percentages.items():
    n = int(TOTAL_UNIQUE_TRANSACTIONS * percentage)
    start, end = span
    for _ in range(n):
        amounts.append(get_random_transaction_amount(start, end+1))
        
random.shuffle(amounts)

In [ ]:
assert len(amounts) == TOTAL_UNIQUE_TRANSACTIONS

In [ ]:
# inspect random sample of transaction amounts
random.sample(amounts, 5)

#### Generate Credit Card Transactions
<br>
<div style="text-align: justify">
Using the random credit card numbers, timestamps and transaction amounts generated in the above steps, 
we can generate random credit card transactions by combining them. The transaction id for the transaction is the md5
hash of the above mentioned entities.
</div>

In [ ]:
def generate_transaction_id(timestamp: str, credit_card_number: str, transaction_amount: float) -> str:
    hashable = f'{timestamp}{credit_card_number}{transaction_amount}'
    hexdigest = hashlib.md5(hashable.encode('utf-8')).hexdigest()
    return hexdigest

In [ ]:
# Use jitter when assigning consumer_ids

def apply_jitter(iterator, increment, max_inc):
    iterator = 0
    increment += 3
    if (increment > max_inc):
        increment = 1
    return iterator, increment

In [ ]:
# Create transactions
transactions = []

MAX_INCREMENT = 99
iter = 0
incr = 1
for timestamp, amount in zip(timestamps, amounts):
    consumer_id = consumer_ids[iter % TOTAL_UNIQUE_CONSUMERS]
    credit_card_number = random.choice(credit_card_numbers)
    transaction_id = generate_transaction_id(timestamp, credit_card_number, amount)
    transactions.append({'tid': transaction_id, 
                         'event_time': timestamp, 
                         'cc_num': credit_card_number, 
                         'consumer_id': consumer_id,
                         'amount': amount, 
                         'fraud_label': 0})
    iter += incr
    if (iter > TOTAL_UNIQUE_CONSUMERS):
        iter, incr = apply_jitter(iter, incr, MAX_INCREMENT)
        

In [ ]:
assert len(transactions) == TOTAL_UNIQUE_TRANSACTIONS

In [ ]:
# inspect random sample of credit card transactions
random.sample(transactions, 3)

### Inject Fradulent Transactions
<p> A typical fraud chain looks like the one as shown in the image below.</p>
<img src="./images/fraud_pattern.png" />

In [ ]:
# FRAUD_RATIO: percentage of transactions that are fraudulent
FRAUD_RATIO = 0.02  # was 0.0025 
NUMBER_OF_FRAUDULENT_TRANSACTIONS = int(FRAUD_RATIO * TOTAL_UNIQUE_TRANSACTIONS)
ATTACK_CHAIN_LENGTHS = [3, 4, 5, 6, 7, 8, 9, 10]

#### Create Transaction Chains 

In [ ]:
visited = set()
chains = defaultdict(list)

In [ ]:
def size(chains: dict) -> int:
    counts = {key: len(values)+1 for (key, values) in chains.items()}
    return sum(counts.values())

In [ ]:
def create_attack_chain(i: int):
    chain_length = random.choice(ATTACK_CHAIN_LENGTHS)
    for j in range(1, chain_length):
        if i+j not in visited:
            if size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS:
                break
            chains[i].append(i+j)
            visited.add(i+j)

In [ ]:
while size(chains) < NUMBER_OF_FRAUDULENT_TRANSACTIONS:
    i = random.choice(range(TOTAL_UNIQUE_TRANSACTIONS))
    if i not in visited:
        create_attack_chain(i)
        visited.add(i)

In [ ]:
assert size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS

#### Modify Transactions with Fraud Chain Attacks 

In [ ]:
def generate_timestamps_for_fraud_attacks(timestamp: str, chain_length: int) -> list:
    timestamps = []
    timestamp = datetime.datetime.strptime(timestamp, ISO_8601_DATETIME_FORMAT)
    for _ in range(chain_length):
        # interval in seconds between fraudulent attacks
        delta = random.randint(30, 120)
        current = timestamp + datetime.timedelta(seconds=delta)
        timestamps.append(current.strftime(ISO_8601_DATETIME_FORMAT))
        timestamp = current
    return timestamps 

In [ ]:
def generate_amounts_for_fraud_attacks(chain_length: int) -> list:
    amounts = []
    for percentage, span in distribution_percentages.items():
        n = math.ceil(chain_length * percentage)
        start, end = span
        for _ in range(n):
            amounts.append(get_random_transaction_amount(start, end+1))
    return amounts[:chain_length]

In [ ]:
for key, chain in chains.items():
    transaction = transactions[key]
    timestamp = transaction['event_time']
    consumer_id = transaction['consumer_id']
    cc_num = transaction['cc_num']
    amount = transaction['amount']
    transaction['fraud_label'] = 1
    inject_timestamps = generate_timestamps_for_fraud_attacks(timestamp, len(chain))
    inject_amounts = generate_amounts_for_fraud_attacks(len(chain))
    random.shuffle(inject_amounts)
    for i, idx in enumerate(chain):
            original_transaction = transactions[idx]
            inject_timestamp = inject_timestamps[i]
            original_transaction['event_time'] = inject_timestamp
            original_transaction['fraud_label'] = 1
            original_transaction['cc_num'] = cc_num
            original_transaction['consumer_id'] = consumer_id
            original_transaction['amount'] = inject_amounts[i]
            original_transaction['tid'] = generate_transaction_id(inject_timestamp, cc_num, amount)
            transactions[idx] = original_transaction

#### Transform Transactions to Pandas DataFrame

In [ ]:
transactions_df = pd.DataFrame(transactions)

In [ ]:
fraud_transactions = transactions_df[transactions_df.fraud_label.eq(1)]
fraud_transactions.head()

In [ ]:
assert fraud_transactions.count()[0] == NUMBER_OF_FRAUDULENT_TRANSACTIONS

### Save Generated Data
<p> The generated raw transactions data will be used by subsequent notebooks. Possible usages include generating aggregates over the raw data columns (using PySpark Windows), or joining these transactions with additional related features (e.g. from a Feature Store). In these ways, we can derive new features which are useful for model training in the later steps. The generated data is saved locally and then copied to S3 bucket.</p>

#### Save Transactions Data to Local Folder ./data and upload to S3

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(data_dir, exist_ok=True)

transactions_df.to_csv(f'{data_dir}/transactions.csv', index=False)

In [ ]:
BASE_PREFIX = "sagemaker-featurestore-blog"

INPUT_KEY_PREFIX = os.path.join(BASE_PREFIX, 'raw')
print (INPUT_KEY_PREFIX)

In [ ]:
transactions_df.to_csv(f's3://{BUCKET}/{INPUT_KEY_PREFIX}/transactions.csv', index=False)